In [2]:
#import the right library's
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #We need this to split the data

In [3]:
#import our dataset which has the number stats of all the players, the dataset we want for training the model. I had to convert the dtype, because not all collumns where the same type.
df = pd.read_csv("PlayerAttributeData.csv", dtype = "a")

In [4]:
#read the position set, this is the thing we want to predict
df2 = pd.read_csv("PlayerPlayingPositionData.csv", dtype = "a")

#we only want the prefferd position collumn from this dataset and of course the id collunm to later merge the datasets
df2 = df2[["Preferred Positions", "Unnamed: 0"]]

In [5]:
#now we merge the two datasets to get our final dataset
df = pd.merge(df, df2, on = "Unnamed: 0")

#to see our final dataset
df.head()

,Unnamed: 0,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,...,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,Preferred Positions
0,0,89,63,89,63,93,95,85,81,91,...,83,94,23,91,92,31,80,85,88,ST LW
1,1,92,48,90,95,95,96,77,89,97,...,88,85,26,87,73,28,59,90,85,RW
2,2,94,56,96,82,95,92,75,81,96,...,81,80,33,90,78,24,53,80,83,LW
3,3,88,78,86,60,91,83,77,86,86,...,83,87,38,77,89,45,80,84,88,ST
4,4,58,29,52,35,48,70,15,14,30,...,55,25,11,61,44,10,83,70,11,GK


I've decided to only use this player data and not the qualitive variables that are in the other data set. Reason being that the most influence for a player position is how good certain skills are. Demographs found in the other dataset will not be more valuable for the model.

In [66]:
#apparently there are some stats in the database which are actually strings with for example 72+2 instead of just 74. 
#So we have to apply a function which fixes this problem. otherwise we can't fit our model, because it doesn't accept strings.
def fix(x):
    x = str(x)
    if('+' in x):
        calc = x.split('+')
        return int(calc[0]) + int(calc[1])
    elif('-' in x):
        calc = x.split('-')
        return int(calc[0]) + int(calc[1])
    elif x.isdigit():
        return int(x)
    else:
         return int(0)
for column in df.loc[:, "Acceleration" : "Volleys"]:
    df[column] = df[column].apply(fix)

#Here we also remove the all the extra posistions, so we just get our main class posistion    
df["Preferred Positions"]= df["Preferred Positions"].str.split(" ", n = 1, expand = True)

In [67]:
#import the randomforst model
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#### Random forest model

I am going to use the random forest algorithm for my model. Random forest is a alhorithm which combines the statistics desicion tree model and combines this with computing power.
The algorithm makes multipile decision trees and decides on a popularity vote from all the trees which classifcation it gets.

My reason for using the random forest tree is pretty simple, i was intrueged by it, because it looks to me that it can make very powerfull predictions. So i just want experience with this model, because of it market standards. Secondly i think this algorithm will get the highest prediction scores (i know it doesn't matter for this exercise, but it matters in the real world)

In [68]:
#split the model in test and train sets so we can do predictions, also create the Y and X values.
X = df[["Sliding tackle", "Finishing", "Crossing", "GK reflexes", "Free kick accuracy", "Curve", "Heading accuracy"]]
y = df["Preferred Positions"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#### Choosing the variables
Note: Normally you want to normalize your X variables so they all can have same influence when predicting. However we don't have to do that on this dataset, because fifa already makes all possible skills on scale from 1 - 100. So everything is already in the same scale.

In a perfect world we would just use all the attribute data in our model, because more data is a better prediction in this case. Every variable gives more insights about what attributes make someone good for a certain desicion. Unfortunately we have to choose 7 now, so what i did was look at the dataset. I then picked the 7 variables that had the most outlying data between 1-100. Because some things like vision, every player has that good till semi-good, so it doens't help the model alot with helping to categorize the different posisitions. While things like tackeling are only skills a good defense player needs, so these attributes will help te model alot. The following variables are the attributes which are only really needed for certain posistions:

1. Sliding tackle
2. Finishing
3. Crossing
4. GK reflexes
5. Free kick accuracy
6. Curve
7. Heading accuracy

In [69]:
#fit our model
clf = RandomForestClassifier(n_estimators=50, max_features = "auto")
clf = clf.fit(X_train, y_train)

In [70]:
#import our evaluation functions
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [71]:
clf.score(X_test, y_test)

0.5086190917516219

#### Evaluating our model (1/2)
Our accuracy is between 0.50861 - 0,51032, which is pretty good. I dabbled around with different settings, but these seem to be the most optimal. Adding trees will increase the score a little bit more. But i have found that from 50 the score only gets improved a insignificent amount. So to keep the preformance steady i choose 50 as our optimal number of trees. I also changed around the max_features, but these would only decrease the score while not really doing something with the speed of the calculations.

In [72]:
#Get per class the the precision, recall and F1-score(which is the balance between recall and precision)
from sklearn.metrics import classification_report
y_test_pred = clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

        CAM       0.29      0.20      0.24       371
         CB       0.70      0.82      0.75       838
        CDM       0.26      0.22      0.24       431
         CF       0.46      0.08      0.13        77
         CM       0.36      0.48      0.41       609
         GK       1.00      1.00      1.00       600
         LB       0.37      0.36      0.37       407
         LM       0.23      0.25      0.24       353
         LW       0.25      0.05      0.08       122
        LWB       1.00      0.04      0.07        28
         RB       0.28      0.30      0.29       319
         RM       0.26      0.22      0.24       411
         RW       0.19      0.04      0.07       137
        RWB       0.00      0.00      0.00        34
         ST       0.66      0.85      0.74       658

avg / total       0.48      0.51      0.48      5395



#### evaluating our model (2/2)
Here we can see the different scores for different predictions. We can see that by far the goalkeeper is de easiest score to predict. Which makes sense because the goalkeeper has speficic skills only needed for goalkeepers. We can see our worst is right wing back, this is because it's not really different from right back and because of this has a really low sample size. We can also see a correaltion between how much instances the class has versus how good it is predicted. Of course this makes sense because those classes just get more data to make more accurate decisions. 

We can also see that for positions which have more players they usually have higher recall, while the more obscure positions have a higher precision. This makes sense because more cases when in doubt get thrown into the more popular positions, causing them to have higher recall, but lower precision.